## Working with ADCP Data
*Written by Friedrich Knuth, Rutgers University, June 12, 2018*

*Revised by Lori Garzio (to work with Pangeo), Rutgers University, July 10, 2018*

In this example we show how to programmatically download and work with 2 Dimensional OOI NetCDF data. We will use data from the ADCP Velocity Profiler (600kHz) deployed at the Coastal Endurance Oregon Inshore Surface Mooring. You will learn:

* how to find the data you are looking for
* how to use the machine to machine API to request data
* how to load the NetCDF data into your notebook, once the data request has completed
* how to explore and plot data

For this example, we are using data from the 600kHaz ADCP Velocity Profiler. You will need the Reference Designator, Stream and Data Delivery Method to make the request to the M2M API. 

More information about the instrument can be found here:
http://ooi.visualocean.net/instruments/view/CE01ISSM-MFD35-04-ADCPTM000

![CE01ISSM-MFD35-04-ADCPTM000](https://github.com/ooi-data-review/ooi_datateam_notebooks/raw/master/images/CE01ISSM-MFD35-04-ADCPTM000.png)

## Request a Dataset

In [ ]:
import requests
import time

Login in at https://ooinet.oceanobservatories.org/ and obtain your <b>API username and API token</b> under your profile (top right corner).

In [ ]:
username = ''
token = ''

Specify your inputs.

In [ ]:
subsite = 'CE01ISSM'
node = 'MFD35'
sensor = '04-ADCPTM000'
method = 'telemetered'
stream = 'adcp_velocity_earth'
beginDT = '2018-03-27T10:32:51.600Z'
endDT = '2018-04-26T10:32:51.600Z'

Request the data. 
#### Note: Data request lines are commented out to prevent accidental resubmission when running through the entire notebook quickly.

In [ ]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,   
}
# r = requests.get(data_request_url, params=params, auth=(username, token))
# data = r.json()

In [ ]:
data['allURLs'][0]

Check if the request has completed.

In [ ]:
%%time
check_complete = data['allURLs'][1] + '/status.txt'
for i in range(1800): 
    r = requests.get(check_complete)
    if r.status_code == requests.codes.ok:
        print('request completed')
        break
    else:
        time.sleep(1)

## Load and Plot the Data

In [ ]:
import re
import os
import xarray as xr

Parse the thredds server to get a list of all NetCDF files. Here, copy and paste the url from *'data['allURLs'][0]'* to avoid having to request the data every time you want to plot.

In [ ]:
# url = data['allURLs'][0]
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180710T142350-CE01ISSM-MFD35-04-ADCPTM000-telemetered-adcp_velocity_earth/catalog.html'
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]
datasets

Load the list of NetCDF files into xarray. Note that this datasets has two dimensions, time and bin. Some variables are dimensioned along just time, others along both time and bin number.

In [ ]:
ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})
ds

Extract the bin depths, east, north and west values along the time dimensions. Note the .T for two dimensional variables.

In [ ]:
time = ds['time'].data
bin_depths = ds['bin_depths'].data.T
east = ds['eastward_seawater_velocity'].data.T
north = ds['northward_seawater_velocity'].data.T
up = ds['upward_seawater_velocity'].data.T

Next we will plot the data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

To screen out surface noise velocities we will set the colorbar limit to the 90th percentile of the data. *Note: np.nanpercentile() is only valid if the data are normally-distributed.*

In [ ]:
lim_east = float("%2.2f" % np.nanpercentile(east, 90))
lim_north = float("%2.2f" % np.nanpercentile(north, 90))
lim_up = float("%2.2f" % np.nanpercentile(up, 90))
u_v_w = max([lim_east, lim_north, lim_up])

And plot.

In [ ]:
plt.close()
fig, axes = plt.subplots(3, sharex=True)
fig.set_size_inches(20, 9)

p0 = axes[0].pcolormesh(time, bin_depths, east, cmap='RdBu',vmin=-u_v_w,vmax=u_v_w)
axes[0].set_ylim(5,25)
axes[0].set_title('Eastward Velocity (u)')
axes[0].invert_yaxis()

p1 = axes[1].pcolormesh(time, bin_depths, north, cmap='RdBu',vmin=-u_v_w,vmax=u_v_w)
axes[1].set_title('Northward Velocity (v)')
axes[1].set_ylim(5,25)
axes[1].invert_yaxis()
axes[1].set_ylabel('Depth')

p2 = axes[2].pcolormesh(time, bin_depths, up, cmap='RdBu',vmin=-u_v_w,vmax=u_v_w)
axes[2].set_title('Upward Velocity (w)')
axes[2].set_ylim(5,25)
axes[2].invert_yaxis()
axes[2].set_xlabel('Time')

fig.colorbar(p0,ax=axes.ravel().tolist(), label='m/s')

plt.show()